# 2. Data handling and visualization

IN GENERAL: INTRODUCE THINGS HERE WHICH WE NEED IN SESSIONS 3-14. THAT MEANS, WHENEVER WE 

### Textbooks & sources

- https://jakevdp.github.io/PythonDataScienceHandbook/
- https://www.pythonlikeyoumeanit.com/index.html

### Notes

- Focus here is on flat data structures (Pandas dataframes) and mathematical data structures (NumPy arrays); hierarchical data structures (JSON and HTML) are covered in session 4.

## 2.1. Essentials

https://www.pythonlikeyoumeanit.com/module_2.html

BOX: OBJECT-ORIENTED PROGRAMING
https://www.pythonlikeyoumeanit.com/module_4.html

## 2.2. Pandas

- https://jakevdp.github.io/PythonDataScienceHandbook/03.00-introduction-to-pandas.html

### 2.2.1. TweetsCOV19 dataset

https://data.gesis.org/tweetscov19/

- Tweet Id: Long.
- Username: String. Encrypted for privacy issues.
- Timestamp: Format ( "EEE MMM dd HH:mm:ss Z yyyy" ).
- #Followers: Integer.
- #Friends: Integer.
- #Retweets: Integer.
- #Favorites: Integer.
- Entities: String. For each entity, we aggregated the original text, the annotated entity and the produced score from FEL library. Each entity is separated from another entity by char ";". Also, each entity is separated by char ":" in order to store "original_text:annotated_entity:score;". If FEL did not find any entities, we have stored "null;".
- Sentiment: String. SentiStrength produces a score for positive (1 to 5) and negative (-1 to -5) sentiment. We splitted these two numbers by whitespace char " ". Positive sentiment was stored first and then negative sentiment (i.e. "2 -1").
- Mentions: String. If the tweet contains mentions, we remove the char "@" and concatenate the mentions with whitespace char " ". If no mentions appear, we have stored "null;".
- Hashtags: String. If the tweet contains hashtags, we remove the char "#" and concatenate the hashtags with whitespace char " ". If no hashtags appear, we have stored "null;".
- URLs: String: If the tweet contains URLs, we concatenate the URLs using ":-: ". If no URLs appear, we have stored "null;"


Download the file https://zenodo.org/record/4593502/files/TweetsCOV19_052020.tsv.gz and store it in the data/tweetscov19/ directory.

In [2]:
import pandas as pd

In [3]:
tweets = pd.read_csv('data/data', sep='\t', header=None)

In [4]:
tweets.columns = ['tweet_id', 'username', 'timestamp', 'followers', 'friends', 'retweets', 'favorites', 'entities', 'sentiment', 'mentions', 'hashtags', 'urls']

In [5]:
tweets

,tweet_id,username,timestamp,followers,friends,retweets,favorites,entities,sentiment,mentions,hashtags,urls
0,1255980348229529601,fa5fd446e778da0acba3504aeab23da5,Thu Apr 30 22:00:24 +0000 2020,29697,24040,0,0,null;,1 -1,null;,Opinion Next2blowafrica thoughts,null;
1,1255981220640546816,547501e9cc84b8148ae1b8bde04157a4,Thu Apr 30 22:03:52 +0000 2020,799,1278,4,6,null;,1 -1,null;,null;,null;
2,1255981244560683008,840ac60dab55f6b212dc02dcbe5dfbd6,Thu Apr 30 22:03:58 +0000 2020,586,378,1,2,null;,2 -1,null;,null;,https://www.bbc.com/news/uk-england-beds-bucks...
3,1255981472285986816,37c68a001198b5efd4a21e2b68a0c9bc,Thu Apr 30 22:04:52 +0000 2020,237,168,0,0,null;,1 -1,null;,null;,https://lockdownsceptics.org/2020/04/30/latest...
4,1255981581354905600,8c3620bdfb9d2a1acfdf2412c9b34e06,Thu Apr 30 22:05:18 +0000 2020,423,427,0,0,i hate u:I_Hate_U:-1.8786140035817729;quaranti...,1 -4,null;,null;,null;
...,...,...,...,...,...,...,...,...,...,...,...,...
1912065,1267207472424660992,ae1b1e6bf2a30cd0e1047ddd0baf5ad0,Sun May 31 21:32:59 +0000 2020,15,45,0,0,spotify:Spotify:-0.9407337067771776;wifi:Wi-Fi...,2 -1,null;,null;,null;
1912066,1267207883487354881,0e4323d01d164b9eb6e33f35564c7e25,Sun May 31 21:34:37 +0000 2020,43,931,0,0,china:China:-2.113921624336916;death penalty:C...,1 -2,null;,null;,null;
1912067,1267209309559173122,00fc2c96e4012e27a6eee351723ab461,Sun May 31 21:40:17 +0000 2020,256,451,0,0,null;,2 -1,null;,null;,null;
1912068,1267212987938545667,0f99a3b8b0d490f062215575d074518b,Sun May 31 21:54:54 +0000 2020,1467,1505,0,0,omg:OMG_%28Usher_song%29:-2.580063760606172;,2 -1,lsddrq,null;,null;


TO IMPLEMENT:
- SPLIT 'sentiment' INTO 'sentiment_pos' AND 'sentiment_neg' AND DELETE THE ORIGINAL SERIES
- CREATE AN 'entities' DF WITH FOUR COLUMNS ['tweet_id', 'original', 'annotated', 'score']
- CREATE A 'mentions' DF WITH ['tweet_id', 'mentionee'] COLUMNS
- CREATE A 'hashtags' DF WITH ['tweet_id', 'hashtag'] COLUMNS
- CREATE A 'urls' DF WITH ['tweet_id', 'url'] COLUMNS

In [6]:
# Splitting 'sentiment' into pos and neg and deleting 'sentiment' column:

pos = []
neg = []

for i in tweets['sentiment']:
    pos.append(i.split()[0])
    neg.append(i.split()[1])
    
tweets['sentiment_pos'] = pos
tweets['sentiment_neg'] = neg

del tweets['sentiment']

tweets

,tweet_id,username,timestamp,followers,friends,retweets,favorites,entities,mentions,hashtags,urls,sentiment_pos,sentiment_neg
0,1255980348229529601,fa5fd446e778da0acba3504aeab23da5,Thu Apr 30 22:00:24 +0000 2020,29697,24040,0,0,null;,null;,Opinion Next2blowafrica thoughts,null;,1,-1
1,1255981220640546816,547501e9cc84b8148ae1b8bde04157a4,Thu Apr 30 22:03:52 +0000 2020,799,1278,4,6,null;,null;,null;,null;,1,-1
2,1255981244560683008,840ac60dab55f6b212dc02dcbe5dfbd6,Thu Apr 30 22:03:58 +0000 2020,586,378,1,2,null;,null;,null;,https://www.bbc.com/news/uk-england-beds-bucks...,2,-1
3,1255981472285986816,37c68a001198b5efd4a21e2b68a0c9bc,Thu Apr 30 22:04:52 +0000 2020,237,168,0,0,null;,null;,null;,https://lockdownsceptics.org/2020/04/30/latest...,1,-1
4,1255981581354905600,8c3620bdfb9d2a1acfdf2412c9b34e06,Thu Apr 30 22:05:18 +0000 2020,423,427,0,0,i hate u:I_Hate_U:-1.8786140035817729;quaranti...,null;,null;,null;,1,-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912065,1267207472424660992,ae1b1e6bf2a30cd0e1047ddd0baf5ad0,Sun May 31 21:32:59 +0000 2020,15,45,0,0,spotify:Spotify:-0.9407337067771776;wifi:Wi-Fi...,null;,null;,null;,2,-1
1912066,1267207883487354881,0e4323d01d164b9eb6e33f35564c7e25,Sun May 31 21:34:37 +0000 2020,43,931,0,0,china:China:-2.113921624336916;death penalty:C...,null;,null;,null;,1,-2
1912067,1267209309559173122,00fc2c96e4012e27a6eee351723ab461,Sun May 31 21:40:17 +0000 2020,256,451,0,0,null;,null;,null;,null;,2,-1
1912068,1267212987938545667,0f99a3b8b0d490f062215575d074518b,Sun May 31 21:54:54 +0000 2020,1467,1505,0,0,omg:OMG_%28Usher_song%29:-2.580063760606172;,lsddrq,null;,null;,2,-1


In [7]:
# Creating entities dataframe:

original = []
annotated = []
score = []

for i in tweets['entities']:
    
    if i == 'null;':
        original.append([''])
        annotated.append([''])
        score.append([''])
    else:
        temp_org = []
        temp_annotated = []
        temp_score = []
        splitted = i.split(';')
        splitted.pop(-1)
        for j in splitted:            
            split = j.split(':')
            temp_org.append(split[0])
            temp_annotated.append(split[1])
            temp_score.append(split[2])
        original.append(temp_org)
        annotated.append(temp_annotated)
        score.append(temp_score)

entities = pd.DataFrame()
entities['tweet_id'] = tweets['tweet_id']
entities['original'] = original
entities['annotated'] = annotated
entities['score'] = score

entities

,tweet_id,original,annotated,score
0,1255980348229529601,[],[],[]
1,1255981220640546816,[],[],[]
2,1255981244560683008,[],[],[]
3,1255981472285986816,[],[],[]
4,1255981581354905600,"[i hate u, quarantine]","[I_Hate_U, Quarantine]","[-1.8786140035817729, -2.3096035868012508]"
...,...,...,...,...
1912065,1267207472424660992,"[spotify, wifi, the playlist]","[Spotify, Wi-Fi, The_Playlist]","[-0.9407337067771776, -2.01842718282239, -2.72..."
1912066,1267207883487354881,"[china, death penalty, drug smuggling]","[China, Capital_punishment, Illegal_drug_trade]","[-2.113921624336916, -2.1414289608078994, -2.3..."
1912067,1267209309559173122,[],[],[]
1912068,1267212987938545667,[omg],[OMG_%28Usher_song%29],[-2.580063760606172]


In [8]:
# creating mentions dataframe:

mentions = pd.DataFrame()
mentions['tweet_id'] = tweets['tweet_id']
mentions['mentions'] = tweets['mentions']
mentions

,tweet_id,mentions
0,1255980348229529601,null;
1,1255981220640546816,null;
2,1255981244560683008,null;
3,1255981472285986816,null;
4,1255981581354905600,null;
...,...,...
1912065,1267207472424660992,null;
1912066,1267207883487354881,null;
1912067,1267209309559173122,null;
1912068,1267212987938545667,lsddrq


In [9]:
# creating hashtags dataframe:

hashtags = pd.DataFrame()
hashtags['tweet_id'] = tweets['tweet_id']
hashtags['hashtags'] = tweets['hashtags']
hashtags

,tweet_id,hashtags
0,1255980348229529601,Opinion Next2blowafrica thoughts
1,1255981220640546816,null;
2,1255981244560683008,null;
3,1255981472285986816,null;
4,1255981581354905600,null;
...,...,...
1912065,1267207472424660992,null;
1912066,1267207883487354881,null;
1912067,1267209309559173122,null;
1912068,1267212987938545667,null;


In [10]:
# creating urls dataframe:

urls = pd.DataFrame()
urls['tweet_id'] = tweets['tweet_id']
urls['urls'] = tweets['urls']
urls

,tweet_id,urls
0,1255980348229529601,null;
1,1255981220640546816,null;
2,1255981244560683008,https://www.bbc.com/news/uk-england-beds-bucks...
3,1255981472285986816,https://lockdownsceptics.org/2020/04/30/latest...
4,1255981581354905600,null;
...,...,...
1912065,1267207472424660992,null;
1912066,1267207883487354881,null;
1912067,1267209309559173122,null;
1912068,1267212987938545667,null;


In [15]:
# Making the first normal form of tweet_id-original dataframe:

import numpy as np

df = pd.DataFrame()
df['tweet_id'] = entities['tweet_id']
df['original'] = entities['original']

lens = list(map(len, df['original'].values))

res = pd.DataFrame({'tweet_id': np.repeat(
    df['tweet_id'], lens), 'original': np.concatenate(df['original'].values)})

res

,tweet_id,original
0,1255980348229529601,
1,1255981220640546816,
2,1255981244560683008,
3,1255981472285986816,
4,1255981581354905600,i hate u
...,...,...
1912067,1267209309559173122,
1912068,1267212987938545667,omg
1912069,1267213122168897541,lagos
1912069,1267213122168897541,mushin


### 2.2.2. Working with a single dataframe

In [15]:
# read/save
# describe()
# changing index and column names
# grouping
# using and resetting the index
# categorize series: categories and codes
# matrix to edgelist and vice versa
# zip
# columns into dict
# datetime
# ...

### 2.2.3. Working with multiple dataframes

In [24]:
# merge split concat etc
# ...

## 2.3. NumPy

- https://jakevdp.github.io/PythonDataScienceHandbook/02.00-introduction-to-numpy.html
- https://www.pythonlikeyoumeanit.com/module_3.html

In [25]:
# read/save
# relationship to pandas
# ...

## 2.4. SciPy

In [1]:
# sparse matrices
# matrix multiplication

## 2.5. Data visualiation with Seaborn & Matplotlib

SUGGESTION: TEACH HOW TO WITH SEABORN, USE MATPLOTLIB WHERE SEABORN DOES NOT OFFER METHODS

- https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html